# Monitoring Neuron Activations

## Motivation 
    - Safety critical systems are interlinked to deep learning.
    - But deep learning models cannot be explicitly extract confident scores
    - Hence we concentrate on implementing the paper[] focused on out of training distribution detection.
    

In [1]:
import pandas as pd
import numpy as np


import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random

from PIL import Image, ImageOps, ImageEnhance
import numbers

import matplotlib.pyplot as plt
%matplotlib inline

# device = 'cuda'

device = 'cpu'

In [2]:
num_classes = 10
num_epochs = 3
batch_size = 1024
learning_rate = 0.01

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data/mnist', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/mnist', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: data/mnist
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
#     device = torch.device('cpu')

device = torch.device('cpu')

In [6]:
# import torch.nn as nn
# import torch.nn.functional as F


# num_classes = 10
# sizeOfNeuronsToMonitor = num_classes*3

# class NeuralNet(nn.Module):
    
#     def __init__(self):
#         super(NeuralNet, self).__init__()
 
#         self.conv1 = nn.Conv2d(1, 40, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(40, 20, 5)
#         self.fc1 = nn.Linear(20 * 4 * 4, 160)
#         self.fc2 = nn.Linear(160, 80)
#         self.fc3 = nn.Linear(80, sizeOfNeuronsToMonitor)
#         self.fc4 = nn.Linear(sizeOfNeuronsToMonitor, num_classes)
        
#     def forward(self, x):
#         # Original 28x28x1 -(conv)-> 24x24x40 -(pool)-> 12x12x40
#         x = self.pool(F.relu(self.conv1(x)))
#         # Original 12x12x40 -(conv)-> 8x8x20 -(pool)-> 4x4x20
#         x = self.pool(F.relu(self.conv2(x)))
#         # Flatten it to an array of inputs
#         x = x.view(-1, 20 * 4 * 4)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         out = self.fc4(x)
#         return out 
  
#     # Here we add another function, which does the same forward computation but also extracts intermediate layer results
#     def forwardWithIntermediate(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 20 * 4 * 4)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         intermediateValues = x
#         x = F.relu(x)
#         out = self.fc4(x)
#         return out, intermediateValues    
    
# net = NeuralNet()
# net.eval()

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        act = out
        out = self.fc1(out)
        out = self.fc2(out)
        return out,act
    
net = ConvNet()
net.eval()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

In [8]:
dict = {}
act_patterns = []
labels_list = []

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
        labels = labels.to(device)
        images = images.to(device)
        net = net.to(device)

            # Forward pass
        outputs = net(images)
        loss = criterion(outputs[0], labels)
        
        act_patterns.append(outputs[1])
        labels_list.append(labels)

            # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


In [10]:
torch.cuda.empty_cache()

In [11]:
with torch.no_grad():
    correct = 0
    outofActivationPattern = 0
    outofActivationPatternAndResultWrong = 0
    
    total = 0
    for images, labels in test_loader:
        labels = labels.to(device)
        images = images.to(device)
        net = net.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs[0], 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.49 %


In [12]:
print(act_patterns)

[tensor([[0.0074, 0.0000, 0.0440,  ..., 0.1270, 0.1899, 0.0786],
        [0.0074, 0.0000, 0.0114,  ..., 0.0808, 0.1666, 0.0567],
        [0.0074, 0.0000, 0.0000,  ..., 0.1284, 0.2106, 0.0625],
        ...,
        [0.0074, 0.0000, 0.0135,  ..., 0.1007, 0.0831, 0.0338],
        [0.0074, 0.0000, 0.0000,  ..., 0.1636, 0.1710, 0.0309],
        [0.0074, 0.0000, 0.0077,  ..., 0.0793, 0.1262, 0.0464]],
       grad_fn=<AsStridedBackward>), tensor([[0.0000, 0.0000, 0.0000,  ..., 0.2688, 0.4166, 0.3416],
        [0.0000, 0.0000, 0.0000,  ..., 0.3197, 0.2176, 0.0557],
        [0.0000, 0.0000, 0.0000,  ..., 0.3593, 0.3559, 0.0828],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2356, 0.3211, 0.3406],
        [0.0000, 0.0000, 0.0000,  ..., 0.3713, 0.3626, 0.0938],
        [0.0000, 0.0000, 0.0000,  ..., 0.3580, 0.3074, 0.0823]],
       grad_fn=<AsStridedBackward>), tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0038, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
 

## References

[1] 